# Process CCP simulation results - Japan microgrid

In [3]:
run_from_collab = False

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform
import pyarrow as pa

%matplotlib inline

In [18]:
# Import custom python file from github repo: https://changhsinlee.com/colab-import-python/
if run_from_collab:
    !pip install requests
    import requests
    # Save python as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/levorato/ccp_rtcs/master/notebooks/rccp_utils.py'
    r = requests.get(url)
    # make sure your filename is the same as how you want to import 
    with open('rccp_utils.py', 'w') as f:
        f.write(r.text)
    # now we can import
    from rccp_utils import *
else:
    from rccp_utils import *

## 1. Process result files

### 1.1. Setup project folders

In [4]:
if run_from_collab:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    gdrive_folder = '/content/gdrive/MyDrive'
else:
    gdrive_folder = '..'
print('gdrive_folder=', gdrive_folder)

gdrive_folder= ..


In [13]:
project_folder = '..'
utc_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(gdrive_folder, "rccp_experiments")
results_folder = os.path.join(gdrive_folder, "rccp_results")
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)

*** Project folder is ..
*** Instances folder is ..\instances
*** Output folder is ..\rccp_experiments


### 1.2. List which experiments to process

In [6]:
experiment_list = ["run_sim_japan_forecast_avg"]

In [7]:
experiment_folder_list = [os.path.join(output_folder, exp) for exp in experiment_list]
experiment_folder_list

['..\\rccp_experiments\\run_sim_japan_forecast_avg']

### 1.3. List which CPP models to process

In [8]:
simulated_model_list = ["robust-budget", "robust-box", "robust-budget"]

### 1.4. Select instance_group to process

In [9]:
instance_group_list = ["japan-10"]

### 1.5. Select RTCS forecast types to process

In [10]:
forecast_type_list = ["average"]  # average-based RTCS forecast

In [14]:
#instances_to_process = ["A_instance2_1000s_skewed-left.txt", "A_instance2_1000s_skewed-right.txt", "A_instance2_1000s_uniform.txt"]
instance_group = "japan-10"
instances_to_process = get_instance_list(project_folder, utc_instances_folder, toy_instances_folder, 
                                         japan_instances_folder, instance_group)
instances_to_process

Processing,  japan-10  instances...
# instances to process:  4


[('..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_autumn.txt',
  '..\\instances\\japan_microgrid\\japan-10\\..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_autumn.txt'),
 ('..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_spring.txt',
  '..\\instances\\japan_microgrid\\japan-10\\..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_spring.txt'),
 ('..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_summer.txt',
  '..\\instances\\japan_microgrid\\japan-10\\..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_summer.txt'),
 ('..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_winter.txt',
  '..\\instances\\japan_microgrid\\japan-10\\..\\instances\\japan_microgrid\\japan-10\\instance_deltamin10_winter.txt')]

In [16]:
df_list = []
for instance_path in instances_to_process:
    instance_name = os.path.basename(instance_path[1])
    result_path = os.path.join(experiment_folder_list[0], "output", "simulation", "zip", instance_name)
    df_ = read_concatenated_trace_df(result_path)
    df_['InstanceName'] = instance_name
    df_list.append(df_)
df = pd.concat(df_list)

Looking for existing simulations in folder ..\rccp_experiments\run_sim_japan_forecast_avg\output\simulation\zip\instance_deltamin10_autumn.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.


ValueError: No objects to concatenate

In [23]:
df.head()

,Model,GammaPerc,Gamma,Strategy,Reoptimize,ModelPolicy,ForecastType,ScenarioId,t,d,ObjValue,OptTimeSpent,e_td,gap,cost,RealProcTime,InstanceName
0,robust-budget,0.0,0.0,conservative,False,ignore_model,average,69,1,1,25984.114879,1.002618,0.0,0.0,106.087870,2.057191,instance_deltamin10_autumn.txt
1,robust-budget,0.0,0.0,conservative,False,ignore_model,average,69,1,2,25984.114879,0.000000,0.0,0.0,104.852168,2.058479,instance_deltamin10_autumn.txt
2,robust-budget,0.0,0.0,conservative,False,ignore_model,average,69,1,3,25984.114879,0.000000,0.0,0.0,119.642430,2.058744,instance_deltamin10_autumn.txt
3,robust-budget,0.0,0.0,conservative,False,ignore_model,average,69,1,4,25984.114879,0.000000,0.0,0.0,122.435927,2.059671,instance_deltamin10_autumn.txt
4,robust-budget,0.0,0.0,conservative,False,ignore_model,average,69,1,5,25984.114879,0.000000,0.0,0.0,123.456134,2.059923,instance_deltamin10_autumn.txt


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12379248 entries, 0 to 143
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Model         object 
 1   GammaPerc     float64
 2   Gamma         float64
 3   Strategy      object 
 4   Reoptimize    bool   
 5   ModelPolicy   object 
 6   ForecastType  object 
 7   ScenarioId    int64  
 8   t             int64  
 9   d             int64  
 10  ObjValue      float64
 11  OptTimeSpent  float64
 12  e_td          float64
 13  gap           float64
 14  cost          float64
 15  RealProcTime  float64
 16  InstanceName  object 
dtypes: bool(1), float64(8), int64(3), object(5)
memory usage: 1.6+ GB


### Create the output folders for processed results

In [17]:
reportfolder = os.path.join(output_folder, 'consolidated_results')
reportfolder_graph = os.path.join(reportfolder, 'graphs')
reportfolder_table = os.path.join(reportfolder, 'tables')
reportfolder_df = os.path.join(reportfolder, 'df')
if not os.path.exists(reportfolder_graph):
    os.makedirs(reportfolder_graph)
if not os.path.exists(reportfolder_table):
    os.makedirs(reportfolder_table)
if not os.path.exists(reportfolder_df):
    os.makedirs(reportfolder_df)
print('Saving files on folder: ' + reportfolder)

Saving files on folder: ..\rccp_experiments\consolidated_results


### Save the consolidated dataframe to file

In [42]:
df.to_pickle(os.path.join(reportfolder_df, experiment_list[0] + '.' + instance_group_list[0] + '.results.pkl.gz'))

## Process Tables

Obtain list of Model, Strategy, ModelPolicy, ForecastType

In [26]:
model_list = df['Model'].unique().tolist()
strategy_list = df['Strategy'].unique().tolist()
model_policy_list = df['ModelPolicy'].unique().tolist()
reoptimize_value_list = df['Reoptimize'].unique().tolist()
forecast_type_list = df['ForecastType'].unique().tolist()
print("Model", model_list)
print("Strategy", strategy_list)
print("ModelPolicy", model_policy_list)
print("Reoptimize", reoptimize_value_list)
print("ForecastType", forecast_type_list)

Model ['robust-budget', 'robust-box', 'deterministic']
Strategy ['conservative', 'audacious', 'cheapest']
ModelPolicy ['ignore_model', 'full_model']
Reoptimize [False, True]
ForecastType ['average']


## Table 1. Simulation performance given all instances 

Model-wise RTCS simulation performance comparison, given all instances.

* Median, Mean, Std. dev and sum of each measure (cost, e_td, gap, time). 

In [27]:
per_instance_stats = dict()
for instance_name in instances_to_process:  # group by instance
    df_itype = df[(df['InstanceName'] == instance_name)]
    _forecast_type_list = df_itype['ForecastType'].unique().tolist()
    for forecast_type in _forecast_type_list:
        df_forecast = df_itype[df_itype['ForecastType'] == forecast_type]
        _model_list = df_forecast['Model'].unique().tolist()
        for model in _model_list:
            df_model = df_forecast[df_forecast['Model'] == model]
            _gamma_perc_list = df_model['GammaPerc'].unique().tolist()
            for gamma_perc in _gamma_perc_list:
                df_gamma = df_model[df_model['GammaPerc'] == gamma_perc]
                _strategy_list = df_gamma['Strategy'].unique().tolist()
                for strategy in _strategy_list:
                    df_strategy = df_gamma[df_gamma['Strategy'] == strategy]
                    _model_policy_list = df_strategy['ModelPolicy'].unique().tolist()
                    for model_policy in _model_policy_list:
                        df_policy = df_strategy[df_strategy['ModelPolicy'] == model_policy]
                        _reoptimize_value_list = df_policy['Reoptimize'].unique().tolist()
                        for reopt in _reoptimize_value_list:
                            df_reopt = df_policy[df_policy['Reoptimize'] == reopt]
                            _scenario_list = df_reopt['ScenarioId'].unique().tolist()
                            for scenario in _scenario_list:
                                df_ = df_reopt[df_reopt['ScenarioId'] == scenario]
                                key = (instance_name, forecast_type, model, gamma_perc, strategy, model_policy, reopt, scenario)
                                per_instance_stats[key] = dict()
                                #per_instance_stats[key]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
                                #per_instance_stats[key]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
                                #per_instance_stats[key]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
                                per_instance_stats[key]['Median time'] = np.round(df_['RealProcTime'].median(), 2)
                                per_instance_stats[key]['Avg. time'] = np.round(df_['RealProcTime'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of time'] = np.round(df_['RealProcTime'].std(), 2)
                                per_instance_stats[key]['Total time'] = np.round(df_['RealProcTime'].sum(), 2)
                                
                                per_instance_stats[key]['Median cost'] = np.round(df_['cost'].median(), 2)
                                per_instance_stats[key]['Avg. cost'] = np.round(df_['cost'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of cost'] = np.round(df_['cost'].std(), 2)
                                per_instance_stats[key]['Total cost'] = np.round(df_['cost'].sum(), 2)
                                
                                per_instance_stats[key]['Median gap'] = np.round(df_['gap'].median(), 2)
                                per_instance_stats[key]['Avg. gap'] = np.round(df_['gap'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of gap'] = np.round(df_['gap'].std(), 2)
                                per_instance_stats[key]['Total gap'] = np.round(df_['gap'].sum(), 2)
                                
                                per_instance_stats[key]['Median e_td'] = np.round(df_['e_td'].median(), 2)
                                per_instance_stats[key]['Avg. e_td'] = np.round(df_['e_td'].mean(), 2)
                                per_instance_stats[key]['Std. dev. of e_td'] = np.round(df_['e_td'].std(), 2)
                                per_instance_stats[key]['Total e_td'] = np.round(df_['e_td'].sum(), 2)

In [28]:
df_table1 = pd.DataFrame.from_dict(per_instance_stats)
df_table1

""


In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### Table 1. Total cost of each simulation, grouped by CCP model and simulation parameters

In [30]:
df.drop(columns=['t', 'd', 'OptTimeSpent']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).sum()

Gamma  \
InstanceName                   Model         GammaPerc Strategy     ModelPolicy  Reoptimize ForecastType              
instance_deltamin10_autumn.txt deterministic 0.0       audacious    full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       cheapest     full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       conservative full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                               robust-box    0.0       audacious    full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       cheapest     full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       conservative full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                               robust-budget 0.0       audacious    full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       cheapest     full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                                       conservative full_model   False      average       0.0         
                                                                                 True       average       0.0         
                                                                    ignore_model False      average       0.0         
                                             20.0      audacious    full_model   False      average       190080.0    
                                                                                 True       average       190080.0    
                                                                    ignore_model False      average       190080.0    
                                                       cheapest     full_model   False      average       190080.0    
                                                                                 True       average       190080.0    
                                                                

### Table 2. Cost of the most expensive scenario (worst simulation cost), grouped by CCP model and simulation parameters

In [31]:
df_t2 = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum().\
    drop(columns=['e_td', 'gap', 'RealProcTime']).\
    groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).\
    max()

df_rob = df_t2.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_t2.reset_index()
df_det = df_det[df_det['Model'] == 'deterministic']
df_wins_t2 = df_rob.merge(df_det, on=['InstanceName', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det', 'Gamma_rob', 'Gamma_det', 'GammaPerc_det'])

In [32]:
df_wins_t2['rob_wins'] = (df_wins_t2['cost_rob'] < df_wins_t2['cost_det']).astype(int)
df_wins_t2.set_index(['InstanceName', 'Model_rob', 'GammaPerc_rob', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType'])


cost_rob  \
InstanceName                   Model_rob     GammaPerc_rob Strategy     ModelPolicy  Reoptimize ForecastType                  
instance_deltamin10_autumn.txt robust-box    0.0           audacious    full_model   False      average       91461.838174    
                               robust-budget 0.0           audacious    full_model   False      average       140636.283227   
                                             20.0          audacious    full_model   False      average       61087.238938    
                                             40.0          audacious    full_model   False      average       40975.456007    
                                             60.0          audacious    full_model   False      average       40233.946210    
                                             80.0          audacious    full_model   False      average       81598.119999    
                                             100.0         audacious    full_model   False      average       50914.866927    
                               robust-box    0.0           audacious    full_model   True       average       91424.426962    
                               robust-budget 0.0           audacious    full_model   True       average       140636.283227   
                                             20.0          audacious    full_model   True       average       60714.613477    
                                             40.0          audacious    full_model   True       average       40957.407009    
                                             60.0          audacious    full_model   True       average       40076.437328    
                                             80.0          audacious    full_model   True       average       81596.540372    
                                             100.0         audacious    full_model   True       average       50890.482210    
                               robust-box    0.0           audacious    ignore_model False      average       91420.711830    
                               robust-budget 0.0           audacious    ignore_model False      average       140636.283227   
                                             20.0          audacious    ignore_model False      average       60714.497900    
                                             40.0          audacious    ignore_model False      average       40952.953563    
                                             60.0          audacious    ignore_model False      average       40060.325336    
                                             80.0          audacious    ignore_model False      average       81589.344595    
                                             100.0         audacious    ignore_model False      average       50877.767796    
                               robust-box    0.0           cheapest     full_model   False      average       91461.838174    
                               robust-budget 0.0           cheapest     full_model   False      average       140636.283227   
                                             20.0          cheapest     full_model   False      average       61087.238938    
                                             40.0          cheapest     full_model   False      average       40975.456007    
                                             60.0          cheapest     full_model   False      average       40233.946210    
                                             80.0          cheapest     full_model   False      average       81598.119999    
                                             100.0         cheapest     full_model   False      average       50914.866927    
                               robust-box    0.0           cheapest     full_model   True       average       91424.426962    
                               robust-budget 0.0           cheapest     full_model   True       average       140636.283227   
                                             20.0  

### Table 3. RTCS performance map (robust wins)

Number of scenarios where Robust RTCS obtained smaller cost, when compared to the Deterministic RTCS, when simulating the same scenario.

In [33]:
df_group = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum()\
    .drop(columns=['gap', 'RealProcTime'])
df_rob = df_group.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_group.reset_index()
df_det = df_det[df_det['Model'] == 'deterministic']
#df_rob
df_wins = df_rob.merge(df_det, on=['InstanceName', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det', 'Gamma_rob', 'Gamma_det', 'GammaPerc_det'])

In [34]:
df_wins['rob_wins'] = (df_wins['cost_rob'] < df_wins['cost_det']).astype(int)

In [35]:
df_wins.head(10)

,InstanceName,Model_rob,GammaPerc_rob,Strategy,ModelPolicy,Reoptimize,ForecastType,ScenarioId,e_td_rob,cost_rob,e_td_det,cost_det,rob_wins
0,instance_deltamin10_autumn.txt,robust-box,0.0,audacious,full_model,False,average,1,0.000000,28677.265131,0.0,26054.470033,0
1,instance_deltamin10_autumn.txt,robust-budget,0.0,audacious,full_model,False,average,1,0.000000,25648.095285,0.0,26054.470033,1
2,instance_deltamin10_autumn.txt,robust-budget,20.0,audacious,full_model,False,average,1,0.000000,28343.329692,0.0,26054.470033,0
3,instance_deltamin10_autumn.txt,robust-budget,40.0,audacious,full_model,False,average,1,0.000000,26932.981461,0.0,26054.470033,0
4,instance_deltamin10_autumn.txt,robust-budget,60.0,audacious,full_model,False,average,1,0.000000,28866.765767,0.0,26054.470033,0
5,instance_deltamin10_autumn.txt,robust-budget,80.0,audacious,full_model,False,average,1,24875.780774,28019.283497,0.0,26054.470033,0
6,instance_deltamin10_autumn.txt,robust-budget,100.0,audacious,full_model,False,average,1,0.000000,27964.217587,0.0,26054.470033,0
7,instance_deltamin10_autumn.txt,robust-box,0.0,audacious,full_model,False,average,2,110161.406099,32349.930002,0.0,26186.785925,0
8,instance_deltamin10_autumn.txt,robust-budget,0.0,audacious,full_model,False,average,2,22981.963379,27401.254866,0.0,26186.785925,0
9,instance_deltamin10_autumn.txt,robust-budget,20.0,audacious,full_model,False,average,2,38477.959051,29698.163874,0.0,26186.785925,0


In [36]:
df_wins.groupby(by=['InstanceName', 'Model_rob', 'GammaPerc_rob', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType']).sum().drop(columns=['ScenarioId'])

e_td_rob  \
InstanceName                   Model_rob     GammaPerc_rob Strategy     ModelPolicy  Reoptimize ForecastType                 
instance_deltamin10_autumn.txt robust-box    0.0           audacious    full_model   False      average       9.940201e+07   
                                                                                     True       average       9.356194e+07   
                                                                        ignore_model False      average       1.384736e+08   
                                                           cheapest     full_model   False      average       9.940201e+07   
                                                                                     True       average       9.356194e+07   
                                                                        ignore_model False      average       1.384736e+08   
                                                           conservative full_model   False      average       8.932186e+07   
                                                                                     True       average       9.110648e+07   
                                                                        ignore_model False      average       1.380353e+08   
                               robust-budget 0.0           audacious    full_model   False      average       1.108080e+08   
                                                                                     True       average       1.019390e+08   
                                                                        ignore_model False      average       1.408509e+08   
                                                           cheapest     full_model   False      average       1.108080e+08   
                                                                                     True       average       1.019390e+08   
                                                                        ignore_model False      average       1.408509e+08   
                                                           conservative full_model   False      average       9.987435e+07   
                                                                                     True       average       9.556128e+07   
                                                                        ignore_model False      average       1.404377e+08   
                                             20.0          audacious    full_model   False      average       4.904754e+07   
                                                                                     True       average       4.463913e+07   
                                                                        ignore_model False      average       5.728795e+07   
                                                           cheapest     full_model   False      average       4.904754e+07   
                                                                                     True       average       4.463913e+07   
                                                                        ignore_model False      average       5.728795e+07   
                                                           conservative full_model   False      average       4.020307e+07   
                                                                                     True       average       3.752558e+07   
                                                                        ignore_model False      average       5.687027e+07   
                                             40.0          audacious    full_model   False      average       9.585363e+05   
                                                                                     True       average       9.627773e+05   
                                                                        ignore_model False      average       9.760973e+05   
                                                           cheapest     full_model

### Table 4. Cheapest RTCS Strategy, per instance and model type

In [37]:
df_group = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType', 'ScenarioId']).sum()\
    .drop(columns=['gap', 'RealProcTime'])
# Find the cheapest strategy for each model type
df_cheapest = df_group.groupby(by=['InstanceName', 'Model', 'GammaPerc', 'ScenarioId']).min().drop(columns=['e_td'])

In [38]:
df_rob = df_cheapest.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_cheapest.reset_index()
df_det = df_det[df_det['Model'] == 'deterministic']
df_wins = df_rob.merge(df_det, on=['InstanceName', 'ScenarioId'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det', 'Gamma_rob', 'Gamma_det', 'GammaPerc_det'])
df_wins['rob_wins'] = (df_wins['cost_rob'] <= df_wins['cost_det']).astype(int)
df_wins['det_wins'] = (df_wins['cost_rob'] > df_wins['cost_det']).astype(int)

In [39]:
df_wins.groupby(by=['InstanceName', 'Model_rob', 'GammaPerc_rob']).sum().drop(columns=['ScenarioId'])

cost_rob  \
InstanceName                   Model_rob     GammaPerc_rob                 
instance_deltamin10_autumn.txt robust-box    0.0            9.676941e+06   
                               robust-budget 0.0            1.217142e+07   
                                             20.0           8.222806e+06   
                                             40.0           7.035038e+06   
                                             60.0           7.030146e+06   
                                             80.0           1.004965e+07   
                                             100.0          7.866635e+06   
instance_deltamin10_spring.txt robust-box    0.0            9.747183e+06   
                               robust-budget 0.0            3.013745e+07   
                                             20.0           8.578252e+06   
                                             40.0           7.914665e+06   
                                             60.0           8.045049e+06   
                                             80.0           7.914718e+06   
                                             100.0          8.572137e+06   
instance_deltamin10_summer.txt robust-box    0.0            1.379902e+07   
                               robust-budget 0.0            1.371598e+07   
                                             20.0           1.382091e+07   
                                             40.0           8.642323e+06   
                                             60.0           1.651251e+07   
                                             80.0           8.641638e+06   
                                             100.0          8.641599e+06   
instance_deltamin10_winter.txt robust-box    0.0            1.402650e+07   
                               robust-budget 0.0            1.947127e+07   
                                             20.0           1.384604e+07   
                                             40.0           1.150215e+07   
                                             60.0           1.304659e+07   
                                             80.0           1.361187e+07   
                                             100.0          9.931227e+06   

                                                                cost_det  \
InstanceName                   Model_rob     GammaPerc_rob                 
instance_deltamin10_autumn.txt robust-box    0.0            7.207465e+06   
                               robust-budget 0.0            7.207465e+06   
                                             20.0           7.207465e+06   
                                             40.0           7.207465e+06   
                                             60.0           7.207465e+06   
                                             80.0           7.207465e+06   
                                             100.0          7.207465e+06   
instance_deltamin10_spring.txt robust-box    0.0            1.042403e+07   
                               robust-budget 0.0            1.042403e+07   
                                             20.0           1.042403e+07   
                                             40.0           1.042403e+07   
                                             60.0           1.042403e+07   
                                             80.0           1.042403e+07   
                                             100.0          1.042403e+07   
instance_deltamin10_summer.txt robust-box    0.0            8.698573e+06   
                               robust-budget 0.0            8.698573e+06   
                                             20.0           8.698573e+06   
                                             40.0           8.698573e+06   
                                             60.0           8.698573e+06   
                                             80.0           8.698573e+06   
                                             100.0          8.698573e+06   
instance_deltamin10_winter.txt robu

### TODO Fazer um box plot dos custos obtidos na simulacao de cada cenario, comparando Rob x Det

### TODO Fazer um kde distribution plot dos custos do RTCS obtidos nas simulacoes: robusto-gamma vs. deterministico

### TODO Fazer uma tabela com as medidas estatisticas (para cada distribuicao usada) de cada simulacao, incluindo valor esperado, SD, percentis 95, 99 e valor maximo observado empiricamente.

In [40]:
for experiment_folder in experiment_folder_list:
    for instance_group in instance_group_list:
        instance_list = get_instance_list(project_folder, antoine_instances_folder, toy_instances_folder, instance_group)
        print(instance_group, instance_list)
        for model in simulated_model_list:
            for forecast_type in forecast_type_list:
                print(model, forecast_type)

TypeError: get_instance_list() missing 1 required positional argument: 'instance_group'